In [ ]:
import pyodbc

# Config db
server = 'server'
database = 'database'
USR = 'USR'
PWD = 'MIS@PWD'
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={USR};PWD={PWD};'

# Old word -> new word
replacements = {
    'tb_old': 'tb_new'
}

# Connect
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

# Proc name
procedure_name = 'dbo.PROC_QUARTIL_GERAL'

try:
    # get the content of proc
    cursor.execute(f"SELECT OBJECT_DEFINITION(OBJECT_ID('{procedure_name}')) AS ROUTINE_DEFINITION")
    procedure_definition = cursor.fetchone().ROUTINE_DEFINITION
    # valid proc
    if procedure_definition is not None:
        # find the as to dont do 'alter inside other alter'
        index_as = procedure_definition.upper().find('AS')

        if index_as != -1:
            new_definition = procedure_definition[index_as + 2:].strip()

            # replace old -> new word
            for old_word, new_word in replacements.items():
                new_definition = new_definition.replace(old_word, new_word)
            
            # alter procedure
            alter_procedure_command = f"""
            ALTER PROCEDURE {procedure_name}
            AS
            {new_definition}
            """

            # commit
            cursor.execute(alter_procedure_command)
            print(f"Sucess")

            connection.commit()

        else:
            print(f"Not fount 'AS' in '{procedure_name}'.")

    else:
        print(f"Stored procedure '{procedure_name}' not found")

except Exception as e:
    print(f"Erro to alter procedure '{procedure_name}': {e}")
    

finally:
    cursor.close()
    connection.close()